In [2]:
import pandas as pd
import sqlite3
import os


In [3]:
folders = ["raw", "processed", "output"]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("✅ Folder structure created successfully")



✅ Folder structure created successfully


In [6]:
raw_file = "/content/retail_sales_dataset.csv"

df = pd.read_csv(raw_file)

print("✅ Dataset Loaded Successfully")
print(df.head())
print("Rows:", df.shape[0], "Columns:", df.shape[1])


✅ Dataset Loaded Successfully
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  
Rows: 1000 Columns: 9


In [7]:
before_rows = df.shape[0]

df.drop_duplicates(inplace=True)
df.fillna(0, inplace=True)

after_rows = df.shape[0]

print("✅ Cleaning Done")
print("Rows before:", before_rows)
print("Rows after :", after_rows)


✅ Cleaning Done
Rows before: 1000
Rows after : 1000


In [8]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

print("✅ Standardized Columns")
print(df.columns)


✅ Standardized Columns
Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount'],
      dtype='object')


In [10]:
df["high_value_order"] = df["total_amount"].apply(lambda x: 1 if x > 500 else 0)

print("✅ Derived Columns Added")
display(df.head())

✅ Derived Columns Added


,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount,high_value_order
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,0
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,0
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,0
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,0


In [11]:
processed_file = "processed/processed_data.csv"
df.to_csv(processed_file, index=False)

print("✅ Processed Data Saved:", processed_file)


✅ Processed Data Saved: processed/processed_data.csv


In [13]:
customers = df[["customer_id"]].drop_duplicates()
customers.to_csv("output/customers.csv", index=False)

In [15]:
orders = df[["transaction_id", "date", "customer_id", "total_amount"]].copy()
orders = orders.rename(columns={
    "transaction_id": "order_id",
    "date": "order_date",
    "total_amount": "sales"
})
orders.to_csv("output/orders.csv", index=False)

In [17]:
products = df[["product_category"]].drop_duplicates()
products.to_csv("output/products.csv", index=False)

print("✅ Customers, Orders, Products split successfully")

✅ Customers, Orders, Products split successfully


In [18]:
conn = sqlite3.connect("database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)

conn.close()

print("✅ Data Loaded into SQLite Successfully")


✅ Data Loaded into SQLite Successfully


In [19]:
print("Validation Report")

print("Customers:", customers.shape[0])
print("Orders   :", orders.shape[0])
print("Products :", products.shape[0])

print("\n✅ ETL Pipeline Completed Successfully")


Validation Report
Customers: 1000
Orders   : 1000
Products : 3

✅ ETL Pipeline Completed Successfully
